In [1]:
!pip install -U -r /projects/Developer/icesat2_boreal/dps/requirements_main.txt

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import os
import glob
import s3fs
s3 = s3fs.S3FileSystem(anon=True)
import multiprocessing

def build_json_mscomp_df(s3_path, glob_str="output*context.json", cols_list=['in_tile_num','max_cloud','start_month_day','end_month_day','start_year','end_year']):
    
    '''Build a data frame of the input multi-spec compositing parameters for each tile'''
    
    dir_tile = os.path.split(s3_path)[0]

    f = s3.glob(os.path.join(dir_tile, glob_str))[0]
    #print(f)
    df = pd.read_json('s3://'+f, typ='series').to_frame().transpose()[cols_list]
    df['json_path'] = s3_path
    
    return df

def build_json_df_TEST(s3_path, glob_str="output*context.json", cols_list=['in_tile_num','max_cloud','start_month_day','end_month_day','start_year','end_year']):
    dir_tile = os.path.split(s3_path)[0]
    #print(os.path.join(dir_tile, glob_str))
    #f = glob.glob(dir_tile + "/" + glob_str)[0]
    f = s3.glob(os.path.join(dir_tile, glob_str))[0]
    #print(f)
    df = pd.DataFrame('s3://'+f).transpose()[cols_list]
    df['json_path'] = s3_path
    return df

In [9]:
INPUT_FN = '/projects/my-public-bucket/DPS_tile_lists/HLS_test_redo/fall2022_check/HLS_tindex_master.csv'
tindex = pd.read_csv(INPUT_FN)
list_paths = tindex.s3_path.to_list()

In [10]:
%%time
df = pd.concat([build_json_mscomp_df(p) for p in list_paths])#.reset_index(drop=True)
df

CPU times: user 1min 1s, sys: 2.62 s, total: 1min 4s
Wall time: 8min 4s


,in_tile_num,max_cloud,start_month_day,end_month_day,start_year,end_year,json_path
0,2822,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...
0,2046,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...
0,3029,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...
0,4112,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...
0,2131,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...
...,...,...,...,...,...,...,...
0,1280,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...
0,1823,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...
0,937,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...
0,742,25,06-15,09-01,2019,2021,s3://maap-ops-workspace/nathanmthomas/dps_outp...


In [136]:
OUTPUT_FN = os.path.join(os.path.dirname(INPUT_FN), 'HLS_input_params.csv')
df.rename(columns={'in_tile_num': 'tile_num'}, inplace=True)
df.to_csv(OUTPUT_FN)
print(f"Wrote MS comp params table: {OUTPUT_FN}")

Wrote MS comp params table: /projects/my-public-bucket/DPS_tile_lists/HLS_test_redo/spring2022/HLS_input_params.csv
